# 선형회귀 예시 (1) - 다이아몬드 가격에 영향을 미치는 요인
## 1. 작업 준비
### 1) 패키지 및 데이터 가져오기
- R의 시각화 패키지인 `ggplot2`에 내장되어 있는 데이터셋으로 총 10개의 요인으로 이루어져 있으며 `53,940`개의 관측치가 있다.

| 변수 | 의미 | 기타 |
|---|---|---|
| carat | 무게 | ex) 3캐럿 다이아몬드 |
| cut | 세공의 질 | 다이아몬드의 단면을 어떻게 커팅하였는지에 대한 상태값을 분류<br/>계층이 있는 범주형(factor) 데이터<br/>"Fair", "Good", "Very Good", "Premium", "Ideal"로 나누어짐 |
| color | 컬러 | 총 7개의 색, 범주형 데이터<br/>
| clarity | 투명도 | 범주형 데이터 |
| depth | 깊이 | 수치형 데이터 |
| table | 넓은 폭 대비 꼭대기의 넓이 | 수치형 데이터 |
| price | 가격(`$`) | 수치형 데이터 |
| x | 길이 | 수치형 데이터 |
| y | 넓이 | 수치형 데이터 |
|  z | 깊이 | 수치형 데이터 |

In [4]:
import sys
sys.path.append('../../')
import helper               # 이렇게 참조할 경우: helper.함수명 으로 코딩해야 함

import numpy as np
from pandas import read_excel, DataFrame, melt, merge
from pca import pca
from pandas.api.types import CategoricalDtype
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from matplotlib import pyplot as plt
import seaborn as sb
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.api import het_breuschpagan
from statsmodels.stats.stattools import durbin_watson

In [5]:
origin = read_excel('https://data.hossam.kr/E04/diamonds.xlsx')
origin

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53935,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


## 2. 데이터 전처리
### 1) 데이터프레임 복사

In [8]:
edf = origin.copy()
edf.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


## 2) 결측치 확인

In [9]:
edf.isna().sum()

carat      0
cut        0
color      0
clarity    0
depth      0
table      0
price      0
x          0
y          0
z          0
dtype: int64

> 결측치 없음

### 3) 데이터 타입 확인

In [10]:
edf.dtypes

carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price        int64
x          float64
y          float64
z          float64
dtype: object

### 4) 일단 분석
#### 1. 회귀분석

In [12]:
tmp = edf.filter(['carat', 'depth', 'table', 'x', 'y', 'z', 'price'])
ols_result = helper.myOls(tmp, y='price',
                          x=['carat', 'depth', 'table', 'x', 'y', 'z'])
print(ols_result.summary)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.859
Model:                            OLS   Adj. R-squared:                  0.859
Method:                 Least Squares   F-statistic:                 5.486e+04
Date:                Thu, 10 Aug 2023   Prob (F-statistic):               0.00
Time:                        10:59:50   Log-Likelihood:            -4.7090e+05
No. Observations:               53940   AIC:                         9.418e+05
Df Residuals:                   53933   BIC:                         9.419e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2.085e+04    447.562     46.584      0.0

> R-squared: 0.859로 설명력이 85%나 되어 좋아보이지만, 이 요인들이 영향을 준다는 결과를 더 확실하게 내기 위해 다른 검정 필요 (r-squared 값이 높아도 실은 이 요인들 중 영향력이 없는 것들이 있을 수 있음)
> Durbin-Watson 값도 확인해야 함